# LLM (Large Language Model) 실습 1

이 노트북은 Transformers 라이브러리를 사용하여 KoGPT2 모델로 한국어 텍스트 생성을 실습합니다.

## 목표
- KoGPT2 모델 로드 및 설정
- 텍스트 생성 파라미터 이해
- GPU/CPU 환경에서의 텍스트 생성 실습

## 1. 라이브러리 임포트

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

## 2. 모델 로드 함수

KoGPT2 모델과 토크나이저를 로드하는 함수입니다.

In [ ]:
def load_model(model_name="skt/kogpt2-base-v2"):
    """모델과 토크나이저를 로드합니다."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

## 3. 디바이스 설정 함수

CUDA가 사용 가능한 경우 GPU를, 그렇지 않으면 CPU를 사용하도록 설정합니다.

In [ ]:
def setup_device(model):
    """디바이스를 설정하고 모델을 해당 디바이스로 이동시킵니다."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"사용 중인 디바이스: {device}")
    return device

## 4. 텍스트 생성 함수

입력된 텍스트를 기반으로 새로운 텍스트를 생성하는 함수입니다.

### 생성 파라미터 설명:
- `max_length`: 생성할 최대 토큰 수
- `repetition_penalty`: 반복을 방지하는 페널티 (높을수록 반복 감소)
- `do_sample`: 샘플링 사용 여부
- `top_k`: top-k 샘플링 (상위 k개 토큰에서만 선택)
- `temperature`: 창의성 조절 (높을수록 더 창의적, 낮을수록 더 보수적)

In [ ]:
def generate_text(tokenizer, model, device, input_text, max_length=50):
    """입력 텍스트를 기반으로 새로운 텍스트를 생성합니다."""
    model.eval()  # 평가 모드로 설정
    
    # 입력 텍스트를 토큰화하고 디바이스로 이동
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    
    # 텍스트 생성
    generated_ids = model.generate(
        input_ids,
        max_length=max_length,  # 최대 생성 길이
        pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 설정
        repetition_penalty=2.0,  # 반복 방지
        do_sample=True,  # 샘플링 활성화
        top_k=50,  # top-k 샘플링
        temperature=0.9  # 온도 설정 (창의성 조절)
    )
    
    # 생성된 토큰을 텍스트로 디코딩
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

## 5. 모델 실행

이제 모든 함수를 사용하여 실제로 텍스트를 생성해보겠습니다.

### 5.1 모델과 토크나이저 로드

In [ ]:
# 모델과 토크나이저 로드
print("모델과 토크나이저를 로드하는 중...")
tokenizer, model = load_model()
print("로드 완료!")

### 5.2 디바이스 설정

In [ ]:
# 디바이스 설정
device = setup_device(model)

### 5.3 텍스트 생성 실행

In [ ]:
# 입력 텍스트 정의
input_text = "안녕하세요. 오늘 날씨"

# 텍스트 생성
print("텍스트 생성 중...")
generated_text = generate_text(tokenizer, model, device, input_text)

# 결과 출력
print("\n" + "="*50)
print("--- 입력 텍스트 ---")
print(input_text)
print("\n--- 생성된 텍스트 ---")
print(generated_text)
print("="*50)

## 6. 다양한 입력으로 실험하기

다양한 입력 텍스트로 실험해보세요.

In [ ]:
# 다양한 입력 텍스트 실험
test_inputs = [
    "인공지능의 미래는",
    "오늘 저녁 메뉴로",
    "코딩을 배우는 것은",
    "여행을 가고 싶은 곳은"
]

for i, input_text in enumerate(test_inputs, 1):
    print(f"\n--- 실험 {i} ---")
    print(f"입력: {input_text}")
    generated = generate_text(tokenizer, model, device, input_text, max_length=60)
    print(f"생성: {generated}")
    print("-" * 40)

## 7. 생성 파라미터 조정 실험

다양한 파라미터 설정으로 텍스트 생성 결과가 어떻게 달라지는지 확인해보세요.

In [ ]:
def generate_text_with_params(tokenizer, model, device, input_text, 
                            max_length=50, temperature=0.9, top_k=50, repetition_penalty=2.0):
    """파라미터를 조정할 수 있는 텍스트 생성 함수"""
    model.eval()
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    
    generated_ids = model.generate(
        input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        top_k=top_k,
        temperature=temperature
    )
    
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# 파라미터 실험
test_input = "인공지능 기술은"

print("=== 파라미터별 생성 결과 비교 ===")
print(f"입력 텍스트: {test_input}\n")

# 낮은 temperature (보수적)
result1 = generate_text_with_params(tokenizer, model, device, test_input, 
                                  temperature=0.3, max_length=60)
print(f"Temperature 0.3 (보수적): {result1}\n")

# 높은 temperature (창의적)
result2 = generate_text_with_params(tokenizer, model, device, test_input, 
                                  temperature=1.2, max_length=60)
print(f"Temperature 1.2 (창의적): {result2}\n")

# 다른 top_k 값
result3 = generate_text_with_params(tokenizer, model, device, test_input, 
                                  top_k=10, max_length=60)
print(f"Top_k 10: {result3}\n")

## 정리

이 실습에서는 다음을 학습했습니다:

1. **모델 로드**: Transformers 라이브러리를 사용한 KoGPT2 모델 로드
2. **디바이스 설정**: GPU/CPU 환경에 따른 적절한 디바이스 설정
3. **텍스트 생성**: 다양한 파라미터를 사용한 텍스트 생성
4. **파라미터 조정**: temperature, top_k 등의 파라미터가 결과에 미치는 영향

### 핵심 개념
- **Temperature**: 생성의 창의성 조절 (낮을수록 보수적, 높을수록 창의적)
- **Top-k sampling**: 상위 k개 토큰에서만 선택하여 품질 향상
- **Repetition penalty**: 반복 방지를 통한 자연스러운 텍스트 생성